In [10]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import datetime
import re
import requests
import pandas as pd

In [11]:
# Класс и атрибут класса возвращающий список с данными с сайта
class Get_list_of_items:
    def __init__(self,types,cl,response):
        self.types,self.cl,self.response = types,cl,response
    def lst(self):
        html = BeautifulSoup(self.response.content, "html.parser")
        titles = html.find_all(self.types, self.cl)
        lst = [i.text for i in html.find_all(self.types, self.cl)]
        return lst
#Функция проверки на соответствие формттата даты
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%B %d, %Y')
    except ValueError:
        raise TypeError('not a date')
#Функция для парсинга данных с метакритика, возвращает датафрейм
def parse_games():
    df = pd.DataFrame(columns=['title','score','platform','date','summary'])
    for page in tqdm(range(200)):
        url = f'https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page={page}'
        response = requests.get(url,headers={'user-agent': 'My app'})
        html = BeautifulSoup(response.content, "html.parser")
        if response.status_code != 200:
            raise TypeError('Response not 200')
        # Получаем список названий
        titles = Get_list_of_items('a', 'title',response)
        lst_titles = titles.lst()
        # Получаем список оценок
        score_wraper = html.find_all('div', class_ = 'clamp-metascore')
        lst_scores = [i.find('div', ['metascore_w large game positive','metascore_w large game mixed','metascore_w large game negative']).text for i in score_wraper]
        # Получаем список платформ
        platforms_row = html.find_all('div', class_ = 'platform')
        lst_platforms = [i.find('span', class_ = 'data').text.replace("\n", "").strip() for i in platforms_row if i.find('span', class_ = 'data') != None]
        # Получаем список дат
        dates = Get_list_of_items('div', 'clamp-details',response)
        lst_dates = [re.search('(?<=\\n\\n)([\w]+[\s]+[\d]+[\,]+[\s][\d]+)', i)[0] for i in dates.lst()]
        next(map(validate, lst_dates))
        # Получаем список описания
        summary = Get_list_of_items('div', 'summary',response)
        lst_summary = [i.replace("\n", "").strip() for i in summary.lst()]
        lst_summary = [re.sub('\[(.*?)\]','', i).strip() for i in lst_summary]
        df1 = pd.DataFrame({'title': lst_titles, 'score': lst_scores, 'platform':lst_platforms,'date':lst_dates,'summary':lst_summary})
        df =pd.concat([df, df1])
    return df.reset_index(drop=True)

In [12]:
df = parse_games()

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [18:47<00:00,  5.64s/it]


In [13]:
df

,title,score,platform,date,summary
0,The Legend of Zelda: Ocarina of Time,99,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ..."
1,Tony Hawk's Pro Skater 2,98,PlayStation,"September 20, 2000",As most major publishers' development efforts ...
2,Grand Theft Auto IV,98,PlayStation 3,"April 29, 2008",What does the American Dream mean today? For N...
3,SoulCalibur,98,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi..."
4,Grand Theft Auto IV,98,Xbox 360,"April 29, 2008",What does the American Dream mean today? For N...
...,...,...,...,...,...
19992,Vroom in the Night Sky,17,Switch,"April 5, 2017",Vroom in the night sky is a magical bike actio...
19993,Leisure Suit Larry: Box Office Bust,17,PlayStation 3,"May 5, 2009",The Leisure Suit Larry: Box Office Bust video ...
19994,Yaris,17,Xbox 360,"October 10, 2007",Hop into a Toyota Yaris and drive it on crazy ...
19995,Ride to Hell: Retribution,16,PC,"June 24, 2013",The game is set in the last years of the roari...
